Install the required libraries

In [ ]:
!pip3 install pandas
!pip3 install plotly
!pip3 install selenium
!pip3 install webdriver-manager

Load the required libraries

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

In [ ]:
# Initialize a webdriver with Chrome engine
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

Create a dataframe with the basic informations for top 250 movies

In [ ]:
# Set the url link for the top 250 movies and load the webpage
url = 'https://www.imdb.com/search/title/?groups=top_250'
driver.get(url)

In [ ]:
# Set up empty lists for each element
indexes = []
titles = []
links = []
years = []
reviews_links = []
genres = []

In [ ]:
# Find and loop through all pages
while True:
    try:
        # Find all movie blocks on the current page
        movie_blocks = driver.find_elements(By.CLASS_NAME, 'lister-item')

        # Extract the movie details on the current page
        for block in movie_blocks:
            # Extract the title
            title_element = block.find_element(By.CLASS_NAME, 'lister-item-header')
            title = title_element.text.strip()
            
            # Extract the index and year from the title
            index = block.find_element(By.CLASS_NAME, 'lister-item-index').text
            year = block.find_element(By.CLASS_NAME, 'lister-item-year').text.strip('()')
            
            # Remove the index and year from the title
            title = title.replace(index, '').replace(year, '').strip().replace("(", "").replace(")", "")

            # Remove the "." from the index
            index = index.replace(".", "")

            # Extract the link
            link_element = title_element.find_element(By.TAG_NAME, 'a')
            link = link_element.get_attribute('href')
            
            # Construct the URL for the reviews page
            review_link = link.replace('?ref_=adv', 'reviews?ref_=tt_urv')

            # Extract the genre
            genre_element = block.find_element(By.CLASS_NAME, 'genre')
            genre = genre_element.text.strip()            
            
            # Append the extracted values to the respective lists
            indexes.append(index)
            titles.append(title)
            links.append(link)
            years.append(year)
            reviews_links.append(review_link)
            genres.append(genre)

        # Go to the next page
        next_button = driver.find_element(By.CLASS_NAME, 'lister-page-next')
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()

        # Wait for a short duration to allow the page to load
        time.sleep(2)  # Adjust the duration as needed

    except NoSuchElementException:
        # If the next button is not found, exit the loop
        break

In [ ]:
# Create a DataFrame from the lists
data = {'Position in IMDB Top 250':indexes, 'Title': titles, 'Year': years, 'Genre':genres, 'Link': links, 'Reviews Link': reviews_links}
df_top_250_movies = pd.DataFrame(data)

In [ ]:
df_top_250_movies

Export Reviews from all the top 250 movies

In [ ]:
# Convert Reviews link column to a list
review_urls = df_top_250_movies["Reviews Link"].values.tolist()

In [ ]:
# Set up empty lists for storing the extracted information
review_titles = []
review_texts = []
review_ratings = []
movie_titles = []

In [ ]:
# Iterate over the list of review URLs
for review_url in review_urls:
    # Visit the reviews page
    driver.get(review_url)

    # Get the page title metadata
    page_title = driver.title

    # Extract the movie title from the page title
    movie_title = page_title.split('(')[0].strip()

    # Keep scrolling and clicking the "Load More" button until it is no longer present
    prev_review_count = 0
    while True:
        try:
            # Find the "Load More" button
            load_more_button = driver.find_element(By.CLASS_NAME, 'ipl-load-more__button')

            # Execute JavaScript to scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Click the "Load More" button using JavaScript
            driver.execute_script("arguments[0].click();", load_more_button)

            # Wait for a short duration to allow the page to load
            time.sleep(2)  # Adjust the duration as needed

            # Find all review elements on the page
            review_elements = driver.find_elements(By.CLASS_NAME, 'review-container')

            # Check the number of loaded reviews
            curr_review_count = len(review_elements)

            # If the number of loaded reviews remains the same, exit the loop
            if curr_review_count == prev_review_count:
                break

            # Update the previous review count
            prev_review_count = curr_review_count

        except NoSuchElementException:
            # If the "Load More" button is not found, exit the loop
            break
    
    # Find all review elements on the page
    review_elements = driver.find_elements(By.CLASS_NAME, 'review-container')

    # Extract the review title, review text, and rating for each review
    for review_element in review_elements:
        # Extract the review title
        title_element = review_element.find_element(By.CLASS_NAME, 'title')
        review_title = title_element.text.strip()

        # Extract the review text
        text_element = review_element.find_element(By.CLASS_NAME, 'text')
        review_text = text_element.text.strip()

        # Extract the rating if available, otherwise assign a default value
        try:
            rating_element = review_element.find_element(By.CLASS_NAME, 'rating-other-user-rating')
            rating = rating_element.text.strip().split('/')[0]  # Extract the rating without the "/10" suffix
        except NoSuchElementException:
            rating = 'N/A'

        # Append the extracted information to the respective lists
        review_titles.append(review_title)
        review_texts.append(review_text)
        review_ratings.append(rating)
        movie_titles.append(movie_title)

In [ ]:
# Create a DataFrame from the extracted information
data = {
    'Review Title': review_titles,
    'Review Text': review_texts,
    'Rating': review_ratings,
    'Movie Title': movie_titles
}
df_reviews_top_250 = pd.DataFrame(data)

# Print the extracted information
print(df_reviews_top_250)

In [ ]:
df_reviews_top_250.to_csv('250_top_movies_reviews.csv')